In [ ]:
import google.generativeai as genai
import os
model = genai.GenerativeModel('gemini-1.5-flash')

GEMINI_API_KEY = "Gemini api key"  # Replace with your actual key

genai.configure(api_key=GEMINI_API_KEY)

#response = model.generate_content("The opposite of hot is")
#print(response.text)


In [ ]:
def negotiate_price(user_message, current_offer, final_offer):
    base_price = 100  # Initial product price
    min_price = 90    # Minimum price the supplier can accept

    # Basic negotiation flow logic based on the user message
    if "yes" in user_message.lower():
        return f"Great! We’ll proceed with the order at ${final_offer}."

    elif "too expensive" in user_message.lower():
        return "I understand. What price would work for you?"

    elif "No, that’s too expensive." in user_message:
        return "I understand. What price would work for you?"

    elif "discount" in user_message.lower():
        return "I can offer you a discount. How does $95 sound?"

    elif "offer" in user_message.lower():
        user_offer = extract_offer(user_message)  # Extract numeric offer
        if user_offer == 90:
            return "Let me check. I can offer you a counteroffer at $95. Would you like to accept?"
        elif user_offer >= base_price:
            return f"Great! We’ll proceed with the order at ${user_offer}."
        elif user_offer >= min_price:
            return f"I see that you're offering ${user_offer}, but the best I can do is $95. Would you like to proceed?"
        else:
            return f"Sorry, I cannot go lower than ${min_price}. Would you be willing to meet at that price?"

    elif "lowest price" in user_message.lower():
        return "The lowest I can go is $90. How does that sound to you?"

    elif "still too high" in user_message.lower():
        return "I understand. How about $90 as a final offer?"

    elif "make it $85" in user_message.lower():
        return "Unfortunately, $85 is below my acceptable range. The lowest I can go is $90. Would that work?"

    elif "finalize" in user_message.lower():
        return f"Great! I'll finalize the order for you at ${final_offer}. Thank you for negotiating with us!"

    else:
        return gemini_generate_response(user_message)  # Fallback to AI model

# Function to extract the price offer from the user's message
def extract_offer(user_message):
    # Find the first number in the message (assuming it’s the offered price)
    words = user_message.split()
    for word in words:
        if word.isdigit():
            return int(word)
    return 0  # Return 0 if no offer found

# Function to call the Gemini API for response generation
def gemini_generate_response(user_message):
    # Generate content using the correct method for Gemini API
    response = model.generate_text(
        prompt=f"You are a supplier negotiating product prices. User: {user_message}"
    )
    # Adjust the response handling based on API format
    return response.result  # This assumes 'result' holds the response text

# Main conversation flow function
def conversation_flow():
    current_offer = 100  # Initial product price
    final_offer = 100    # This will track the final negotiated price
    print(f"Bot: Welcome! The price for this product is ${current_offer}. Would you like to proceed with this price, or make an offer?")

    while True:
        user_message = input("You: ")

        if "no" in user_message.lower() and "too expensive" not in user_message.lower():
            print("Bot: Thank you for your time. Feel free to reach out if you change your mind.")
            break

        elif "changed my mind" in user_message.lower():
            print("Bot: No worries! Let me know if you need any assistance in the future.")
            break

        else:
            bot_response = negotiate_price(user_message, current_offer, final_offer)
            print(f"Bot: {bot_response}")

            # Update final offer if user accepts an offer
            if "Great! We’ll proceed with the order at" in bot_response:
                final_offer = extract_offer(bot_response)  # Update final offer based on response

            # Handle finalization or proceed logic
            if "Great! I'll finalize the order" in bot_response:
                break
            if "proceed" in bot_response.lower():
                break

# Run the chatbot conversation
conversation_flow()

Bot: Welcome! The price for this product is $100. Would you like to proceed with this price, or make an offer?
Bot: I understand. What price would work for you?
Bot: I can offer you a discount. How does $95 sound?
Bot: Sorry, I cannot go lower than $90. Would you be willing to meet at that price?
